In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob

In [ ]:
fnames = glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T1.0_P0.*.lammpstrj')
#fnames = fnames + glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T0.700_P0.0000020000.lammpstrj')
#fnames = glob('/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/LJ_CRYSTAL/T_0.700_box/dumplin/dump.npt_nose_T1.0_P0.0000000000.lammpstrj')
arrays = [np.loadtxt(f, skiprows=9)[:,2:5] for f in fnames]
coordinates = np.array(arrays)
temperature = float(1.00)
side=3.
dim = len(coordinates)
n_particles = len(coordinates[0])
spacial_dim = 3
dim_ics = n_particles * spacial_dim
coordinates=coordinates.reshape(len(arrays), dim_ics)

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

# a context tensor to send data to the right device and dtype via '.to(ctx)'
ctx = torch.zeros([], device=device, dtype=dtype)

# a brief check if this module is the main executable (or imported)
main = (__name__ == "__main__")

In [ ]:
import sys
sys.path.insert(0, "/home/ninarell/OneDrive/WF_GAN_FOR_GLASSES/B_GEN/bgflow")
import bgflow as bg

In [ ]:
from  bgflow.distribution.energy import LennardJonesPotentialPPP
rm = 2**(1./6.)
target = LennardJonesPotentialPPP(dim = dim_ics, n_particles = n_particles, side = side, oscillator = False, rm=rm, two_event_dims=False)

In [ ]:
def plot_energy(coordinates, target):
    xs = torch.Tensor(coordinates)
    #xs = xs.view(-1,10,3)
    energy = target.energy(xs).detach().numpy()
    x=np.arange(1,len(energy)+1)

    fig = plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, energy)

    plt.subplot(1, 2, 2)
    counts, bins = np.histogram(energy, density=True)
    plt.yscale("log")
    plt.stairs(counts, bins)

In [ ]:
def save_energy(coordinates, target, step):
    xs = torch.Tensor(coordinates)
    #xs = xs.view(-1,10,3)
    energy = target.energy(xs).detach().numpy()
    x=np.arange(1,len(energy)+1)

    fig = plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, energy)

    plt.subplot(1, 2, 2)
    counts, bins = np.histogram(energy, density=True)
    plt.yscale("log")
    plt.stairs(counts, bins)
    
    filename = '/home/ninarell/Desktop/FIG_ENERGY/fig_'+str(step)+".png"
    fig.savefig(filename, dpi=fig.dpi, format='png')
    plt.close()

In [ ]:
plot_energy(coordinates, target)

In [ ]:
from bgflow import NormalDistribution, TruncatedNormalDistribution

mean = torch.Tensor(coordinates[50]).view(dim_ics)
cov = torch.zeros(dim_ics, dim_ics)
cov = cov.fill_diagonal_(0.00002)

prior = NormalDistribution(dim_ics, mean=mean, cov=cov)

#upper_bound=torch.ones(dim_ics)*side
#lower_bound=-upper_bound
#sigma=torch.ones(dim_ics)*0.002
#print(upper_bound.shape, lower_bound.shape, sigma.shape, mean.shape )
#prior = TruncatedNormalDistribution(mu=mean, sigma=sigma, lower_bound=lower_bound, upper_bound=upper_bound)

In [ ]:
data_prior = prior._sample_with_temperature(10000, temperature=temperature)
#data_prior = data_prior.view(-1, 10, 3)
plot_energy(data_prior, target)

In [ ]:
# define a MCMC sampler to sample from the target energy

from bgflow import GaussianMCMCSampler, LatentGaussianMCMCSampler

init_state = torch.Tensor(coordinates[10])
init_state.shape

In [ ]:
def constraint(x):
    return torch.where((abs(x) > side).clone().detach(), x - 2 * side * torch.sign(x) * torch.floor(abs(x)/side) , x)

In [ ]:
from bgflow.utils.types import is_list_or_tuple
from bgflow.utils.types import assert_numpy

class LossReporter:
    """
        Simple reporter use for reporting losses and plotting them.
    """
    
    def __init__(self, *labels):
        self._labels = labels
        self._n_reported = len(labels)
        self._raw = [[] for _ in range(self._n_reported)]
    
    def report(self, *losses):
        assert len(losses) == self._n_reported
        for i in range(self._n_reported):
            self._raw[i].append(assert_numpy(losses[i]))
    
    def plot(self, n_smooth=10):
        fig, axes = plt.subplots(self._n_reported, sharex=True)
        if not isinstance(axes, np.ndarray):
            axes = [axes]
        fig.set_size_inches((8, 4 * self._n_reported), forward=True)
        for i, (label, raw, axis) in enumerate(zip(self._labels, self._raw, axes)):
            raw = assert_numpy(raw).reshape(-1)
            kernel = np.ones(shape=(n_smooth,)) / n_smooth
            smoothed = np.convolve(raw, kernel, mode="valid")
            axis.plot(smoothed)
            axis.set_xscale("log")
            axis.set_ylabel(label)
            if i == self._n_reported - 1:
                axis.set_xlabel("Iteration")
                
    def recent(self, n_recent=1):
        return np.array([raw[-n_recent:] for raw in self._raw])

In [ ]:
# define a flow with RNVP coupling layers

from bgflow.nn import (
    DenseNet,
    SequentialFlow, 
    CouplingFlow, 
    AffineFlow, 
    SplitFlow, 
    InverseFlow, 
    SwapFlow,
    AffineTransformer
)



# here we aggregate all layers of the flow
layers = []

# start with a splitting layer which splits the input tensor into two 
# flow channels with tensors of half dimensionality
layers.append(SplitFlow( dim_ics//2 , dim=1))
#layers.append(SplitFlow( n_particles//2 , dim=1))

# now add coupling layers
n_coupling_layers = 4
for _ in range(n_coupling_layers):
    
    # we need to swap dimensions for the mixing
    layers.append(SwapFlow())
    
    # now set up a coupling block
    layers.append(CouplingFlow(
        # we use a affine transformation to transform the RHS conditioned on the LHS
        AffineTransformer(
            # use simple dense nets for the affine shift/scale
            shift_transformation=DenseNet([dim_ics//2, 100, 100, dim_ics//2], activation=torch.nn.ReLU()),
            scale_transformation=DenseNet([dim_ics//2, 100, 100, dim_ics//2], activation=torch.nn.ReLU())
            #shift_transformation=DenseNet([n_particles//2, 100, 100, n_particles//2], activation=torch.nn.ReLU()),
            #scale_transformation=DenseNet([n_particles//2, 100, 100, n_particles//2], activation=torch.nn.ReLU())
        )
    ))
    
# finally, we have to merge the two channels again into one tensor
layers.append(InverseFlow(SplitFlow( dim_ics//2 , dim=1)))
    
# now define the flow as a sequence of all operations stored in layers
flow = SequentialFlow(layers)

In [ ]:
# having a flow and a prior, we can now define a Boltzmann Generator

from bgflow import BoltzmannGenerator

bg = BoltzmannGenerator(prior, flow, target)

In [ ]:
#bg._flow[8].transformer._shift_transformation._layers

In [ ]:
sample_before = bg.sample(500, temperature=temperature)
plot_energy(sample_before , target)

In [ ]:
# initial training with likelihood maximization on data set

from bgflow.utils.train import IndexBatchIterator

n_batch = 10
batch_iter = IndexBatchIterator(len(data_prior), n_batch)

optim = torch.optim.Adam(bg.parameters(), lr=5e-3)

n_epochs = 3
n_report_steps = 50

reporter = LossReporter("NLL")

In [ ]:
for epoch in range(n_epochs):
    for it, idxs in enumerate(batch_iter):
        #print(it, idxs, len(data))

        batch = data_prior[idxs]
        #print(idxs, batch, "\n\n")
        optim.zero_grad()
        
        # negative log-likelihood of the batch is equal to the energy of the BG
        nll = bg.energy(batch, temperature=temperature).mean()
        nll.backward()
        
        reporter.report(nll)
        
        optim.step()
        
        if it % n_report_steps == 0:
            print("\repoch: {0}, iter: {1}/{2}, NLL: {3:.4}".format(
                    epoch,
                    it,
                    len(batch_iter),
                    *reporter.recent(1).ravel()
                ), end="")
        


In [ ]:
reporter.plot()

In [ ]:
sample = bg.sample(500, temperature=temperature)

In [ ]:
plot_energy(sample, target)

In [ ]:
from bgflow import LatentGaussianMCMCSampler

In [ ]:
# train with convex mixture of NLL and KL loss

from bgflow.utils.train import IndexBatchIterator

n_kl_samples = 10
n_batch = 10
batch_iter = IndexBatchIterator(len(data_prior), n_batch)

optim = torch.optim.Adam(bg.parameters(), lr=3e-6)

n_epochs = 10
n_report_steps = 5

# mixing parameter
lambdas = torch.linspace(1., 0.5, n_epochs)

reporter = LossReporter("NLL", "KLL")

In [ ]:
for epoch, lamb in enumerate(lambdas):
    for it, idxs in enumerate(batch_iter):
        batch = data_prior[idxs]
        
        optim.zero_grad()
        
        # negative log-likelihood of the batch is equal to the energy of the BG
        nll = bg.energy(batch).mean()
        
        # aggregate weighted gradient
        (lamb * nll).backward()
        
        # kl divergence to the target
        kll = bg.kldiv(n_kl_samples).mean()

        # aggregate weighted gradient
        ((1. - lamb) * kll).backward()
        
        reporter.report(nll, kll)
        
        optim.step()
        
        if it % n_report_steps == 0:
            print("\repoch: {0}, iter: {1}/{2}, lambda: {3}, NLL: {4:.4}, KLL: {5:.4}".format(
                    epoch,
                    it,
                    len(batch_iter),
                    lamb,
                    *reporter.recent(1).ravel()
                ), end="")
            
            
        for i in range(len(batch)):
            sample  = batch[i:i+1]
            latent_sampler = LatentGaussianMCMCSampler(target, flow=flow, init_state=sample,temperature=temperature, noise_std=0.01)
            for j in range(1):
                 sample = latent_sampler.sample(1)
                    
            if torch.any(sample.isnan()):
                print(sample, batch[i:i+1])
            real_sample = bg.sample(1, temperature=temperature)
            if torch.any(real_sample.isnan()):
                print(sample, batch[i:i+1], real_sample)
                
            batch[i:i+1] = sample
        data_prior[idxs] = batch
    samples = bg.sample(2000, temperature=temperature)
    save_energy(samples, target, str(epoch)+"_"+str(lamb))
    

In [ ]:
reporter.plot()

In [ ]:
samples = bg.sample(2000, temperature=temperature)
plot_energy(samples,target)